In [0]:
import random
import math
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score,confusion_matrix,classification_report

In [0]:
w = 0.729844 
c1 = 1.496180 
c2 = 1.496180 

In [0]:
def model_score(x_train, y_train, x_test, y_test, classes = 'multi'):
    
    #x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
    model = DecisionTreeClassifier(random_state = 101)
    #print(type(model))
    model.fit(x_train,y_train)
    
    test_predictions = model.predict(x_test)
    if classes == 'multi':
        return f1_score(y_test, test_predictions, average = 'macro')
    else:
        return f1_score(y_test, test_predictions)

In [0]:
def crossover(mother, father):

    child = [None]*len(father)
    for param in range(len(father)):
        child[param] = random.choice([mother[param], father[param]])

    return child


In [0]:
class Particle:
     
    def __init__(self,dimension):
        self.velocity = []
        self.pos = []
        self.pBest = []
        self.dimension = dimension
        n_chosen = random.randint(1,self.dimension-1)
        features = [0]*self.dimension
        for i in range(n_chosen):
            feat = random.randint(0,self.dimension-1)
            features[feat] = 1
        for i in range(self.dimension):
            self.pos.append(features[i])
            self.velocity.append(0.01*random.random())
            self.pBest.append(self.pos[i])
        #print(len(self.pos))
        return
 
    def updatePositions(self,gBest, t, maxIter):
        for i in range(self.dimension):
            
            temp=1/(1+math.exp(-self.velocity[i]))
            if temp >= 0.5:
                self.pos[i]=1
            else:
                self.pos[i]=0
        cross_child=crossover(self.pos,gBest)
        return
 
    def updateVelocities(self, gBest):
        for i in range(self.dimension):
            r1 = random.random()
            r2 = random.random()
            social = c1 * r1 * (gBest[i] - self.pos[i])
            cognitive = c2 * r2 * (self.pBest[i] - self.pos[i])
            self.velocity[i] = (w * self.velocity[i]) + social + cognitive
            
        return 
   

In [0]:
class ParticleSwarmOptimizer:
     
    def __init__(self, X_train, y_train, X_test, y_test, swarmsize = 15):
        #self.solution = []
        self.swarm = []
        self.fitness_score=0
        self.f1_score=0
        self.alpha = 0.6
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        self.swarmSize = swarmsize
        for h in range(self.swarmSize):
            particle = Particle(dimension=X_train.shape[1])
            self.swarm.append(particle)
        return
 
    def optimize(self, iterations = 10, classes = 'multi'):
        for i in range(iterations):
           
            self.gBest = self.swarm[0].pos
        
            
            for j in range(self.swarmSize):
                pBest = self.swarm[j].pBest
                pBScore=self.f(pBest, classes = classes)
                gBScore=self.f(self.gBest, classes = classes)
                if pBScore[0] > gBScore[0]:
                    self.gBest = pBest
                    self.fitness_score = pBScore[0]
                    self.f1_score = pBScore[1]
            
            no_of_features = sum(self.gBest)
          
            for k in range(self.swarmSize):
                self.swarm[k].updateVelocities(self.gBest)
                self.swarm[k].updatePositions(self.gBest,i,iterations)
    
            for l in range(self.swarmSize):
                
                pBest = self.swarm[l].pBest
                pBScore = self.f(pBest, classes = classes)
                currfitness = pBScore[0]
                currf1 = pBScore[1]
                
                currPBest=self.f(self.swarm[l].pos, classes = classes)
                
                if currPBest[0] > currfitness:
                    self.swarm[l].pBest = self.swarm[l].pos
                    currfitness=currPBest[0]
                    currf1=currPBest[1]
                    
        return no_of_features,self.f1_score, self.gBest
 
    def f(self,feature_array, classes):
         
        feature_array = [round(i) for i in feature_array]
        feature_index = [i for i in range(len(feature_array)) if feature_array[i] > 0]
        f1_score = model_score(self.X_train[:,feature_index], self.y_train, self.X_test[:,feature_index], self.y_test, classes = classes)
        fitness_score = self.alpha*f1_score + (1-self.alpha)*(X_train.shape[1]-len(feature_index))/X_train.shape[1]
        return [fitness_score,f1_score]
        

In [0]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import numpy as np

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
X_train=np.load("/content/drive/My Drive/Major2/X_train.npy")
y_train=np.load("/content/drive/My Drive/Major2/y_train.npy")
X_test=np.load("/content/drive/My Drive/Major2/X_test.npy")
y_test=np.load("/content/drive/My Drive/Major2/y_test.npy")

In [0]:
X_train = X_train.reshape(840, 208*176)
X_test = X_test.reshape(360, 208*176)

In [0]:
X_train.shape

(840, 36608)

In [0]:
pso = ParticleSwarmOptimizer(X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test)

In [0]:
number_of_features, f1_score, feature_index = pso.optimize()

iteration  0
[0.8663371952064222, 1667, 0.8075863218475334]
~~~~~~~
[0.7331885696038921, 16358, 0.8532090961929671]
[0.7847775714722931, 13186, 0.8814256144584838]
[0.8663371952064222, 1667, 0.8075863218475334]
[0.7536276848151847, 13925, 0.8429670329670329]
[0.7985254715806187, 9745, 0.8416748827042945]
[0.7987451784374532, 8435, 0.8181847088176668]
[0.8127135588015311, 8443, 0.8416110304034841]
[0.8011298732302923, 9191, 0.8359266826565478]
[0.865931752554179, 3632, 0.8426951120658228]
[0.7724515308927891, 11571, 0.8314715200194502]
[0.6939493908438661, 18242, 0.8221199054856976]
[0.7117792759954443, 17676, 0.8415289798059271]
[0.7459424186856642, 13885, 0.8294298178560604]
[0.7702453750037344, 11647, 0.829178626171725]
[0.7323576862715291, 18389, 0.8888107416879796]
iteration  1
[0.8663371952064222, 1667, 0.8075863218475334]
~~~~~~~
[0.7331885696038921, 16358, 0.8532090961929671]
[0.7847775714722931, 13186, 0.8814256144584838]
[0.8663371952064222, 1667, 0.8075863218475334]
[0.753627

In [0]:
print(feature_index)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 

In [0]:
len(feature_index)

36608

In [0]:
indices= [i for i,v in enumerate(feature_index) if v > 0]

In [0]:
len(indices)

1667

In [0]:
print(indices)

[67, 85, 95, 96, 114, 156, 196, 198, 217, 231, 238, 326, 336, 338, 371, 372, 379, 390, 393, 406, 463, 492, 502, 509, 513, 523, 533, 539, 542, 546, 580, 618, 624, 646, 651, 680, 701, 706, 738, 753, 766, 810, 817, 818, 883, 898, 903, 919, 929, 994, 1008, 1015, 1064, 1087, 1090, 1097, 1101, 1117, 1130, 1185, 1208, 1323, 1332, 1333, 1339, 1343, 1440, 1446, 1483, 1506, 1558, 1572, 1604, 1632, 1651, 1654, 1657, 1697, 1701, 1722, 1745, 1797, 1802, 1813, 1821, 1845, 1909, 1910, 1940, 1979, 2014, 2026, 2052, 2081, 2089, 2097, 2104, 2113, 2129, 2161, 2193, 2285, 2296, 2318, 2368, 2374, 2397, 2412, 2428, 2454, 2496, 2521, 2544, 2562, 2612, 2622, 2662, 2672, 2676, 2744, 2766, 2773, 2796, 2836, 2848, 2874, 2885, 2913, 2937, 2951, 2953, 2958, 2966, 2975, 3011, 3025, 3061, 3159, 3167, 3193, 3195, 3197, 3223, 3394, 3424, 3433, 3460, 3502, 3511, 3525, 3588, 3603, 3607, 3638, 3670, 3685, 3696, 3702, 3707, 3734, 3737, 3744, 3748, 3755, 3769, 3776, 3793, 3798, 3802, 3804, 3819, 3842, 3846, 3856, 3859, 386

In [0]:
X_train.shape

(840, 36608)

In [0]:
X_train_PG=X_train[:,indices]
X_test_PG=X_test[:,indices]

In [0]:
X_train_PG.shape

(840, 1667)

In [0]:
np.save("X_train_PG3.npy",X_train_PG)
np.save("y_train_PG3.npy",y_train)
np.save("X_test_PG3.npy",X_test_PG)
np.save("y_test_PG3.npy",y_test)

In [0]:
from google.colab import files
files.download('X_train_PG3.npy')
files.download('X_test_PG3.npy')
files.download('y_train_PG3.npy')
files.download('y_test_PG3.npy')